# Feature Engineering
This notebook outlines the steps taken to prepare the data for modeling
The goal is to have the following dataframe format:

* Columns for Price features on the day of the trades we are analyzing (Input variables)
* Columns for Future prices at different dates (Response variables)

In [17]:
import pandas as pd
import numpy as np

### Import quote & trade datasets

In [2]:
url = "https://raw.githubusercontent.com/AdrianGPrado/StockMarket-ML/CK/all_quotes_features.csv"
quotesAll = pd.read_csv(url,index_col=0,parse_dates=[0], sep='\t', encoding='utf-8')
url = "https://raw.githubusercontent.com/AdrianGPrado/StockMarket-ML/CK/trades.csv"
trades = pd.read_csv(url,index_col=0,parse_dates=[0], sep=',', encoding='utf-8')

We will be joining our trade and quotes datasets as our features (technical indicators) will be derived from the time of the trades.

In [3]:
#RESET INDEXES & CREATE 'TICKER' COLUMNS
quotesAll['TradeDate'] = quotesAll.index
quotesAll = quotesAll.set_index(['Ticker'])
quotesAll['Ticker'] = quotesAll.index
trades['Ticker'] = trades.index

#APPLY COMMON DATE FORMATTING TO 'TRADEDATE' COLUMNS
quotesAll['TradeDate'] = quotesAll['TradeDate'].dt.date
trades['TradeDate'] = pd.to_datetime(trades['TradeDate'], errors='coerce')
trades['TradeDate'] = trades['TradeDate'].dt.date

### Lets look at the datasets

In [4]:
trades.head()

,Strike,OptionType,ActivityType,InitialVolume,IS_Flag,TweetTimeStamp,TradeDate,TradeTime,ExpDate,startOpen,startLow,startHigh,startClose,startDayDelt,EarningsDate,EarningsTime,Ticker
Ticker,,,,,,,,,,,,,,,,,
GGP,29.0,Calls,BUYING,1300,NaN,2016-07-12 14:59:17,2016-07-12,14:59:17,2016-07-15,31.09,30.48,31.18,31.06,-0.000965,2016-08-01,after,GGP
SPY,218.0,Calls,SELLING,2503,NaN,2016-07-12 14:58:51,2016-07-12,14:58:51,2016-08-05,214.53,213.43,215.30,214.95,0.001958,NaN,NaN,SPY
JNPR,22.0,Calls,BUYING,493,NaN,2016-07-12 14:39:41,2016-07-12,14:39:41,2016-07-15,23.05,22.97,23.29,23.10,0.002169,2016-07-26,after,JNPR
RLGY,30.0,Calls,BUYING,500,NaN,2016-07-12 14:38:01,2016-07-12,14:38:01,2016-08-19,29.34,29.33,29.94,29.71,0.012611,2016-08-04,before,RLGY
GLD,127.0,Calls,BUYING,10000,NaN,2016-07-12 14:29:40,2016-07-12,14:29:40,2016-07-15,128.52,126.99,128.54,127.15,-0.010660,NaN,NaN,GLD


In [5]:
quotesAll.head()

,100d,200d,21d,50d,9d,Adj Close,Close,Date,High,Low,MACD,Open,RSI,Volume,lower_band,max,min,upper_band,TradeDate,Ticker
Ticker,,,,,,,,,,,,,,,,,,,,
GGP,NaN,NaN,NaN,NaN,NaN,28.210024,29.549999,NaN,29.559999,28.860001,0.000000,28.910000,NaN,6466600,NaN,NaN,NaN,NaN,2015-11-02,GGP
GGP,NaN,NaN,NaN,NaN,NaN,27.398570,28.700001,NaN,29.080000,28.420000,-0.018206,28.570000,NaN,9076300,NaN,NaN,NaN,NaN,2015-11-03,GGP
GGP,NaN,NaN,NaN,NaN,NaN,27.379476,28.680000,NaN,28.830000,28.559999,-0.023790,28.719999,NaN,3623100,NaN,NaN,27.379476,NaN,2015-11-04,GGP
GGP,NaN,NaN,NaN,NaN,NaN,27.532221,28.840000,NaN,29.049999,28.570000,-0.020232,28.670000,NaN,4137500,NaN,27.532221,NaN,NaN,2015-11-05,GGP
GGP,NaN,NaN,NaN,NaN,NaN,26.252984,27.500000,NaN,28.420000,27.320000,-0.068776,28.150000,NaN,6539300,NaN,NaN,NaN,NaN,2015-11-06,GGP


### Join Data Frames

In [6]:
# JOIN DATA FRAMES
quotesAll_join = pd.merge(quotesAll, trades, how='left', on=['TradeDate','Ticker'])

### Create Future Price (Response) Columns

In [7]:
# CREATE FUTURE PRICE (RESPONSE) COLUMNS
## Also bring in the associate Tickers with the future prices
## This will serve to exclude mismatched Ticker rows later
quotesAll_join['5DayFuturePrice'] = quotesAll_join['Adj Close'].shift(-5)
quotesAll_join['5DayFutureTicker'] = quotesAll_join['Ticker'].shift(-5)

quotesAll_join['10DayFuturePrice'] = quotesAll_join['Adj Close'].shift(-10)
quotesAll_join['10DayFutureTicker'] = quotesAll_join['Ticker'].shift(-10)

quotesAll_join['20DayFuturePrice'] = quotesAll_join['Adj Close'].shift(-20)
quotesAll_join['20DayFutureTicker'] = quotesAll_join['Ticker'].shift(-20)

quotesAll_join['30DayFuturePrice'] = quotesAll_join['Adj Close'].shift(-30)
quotesAll_join['30DayFutureTicker'] = quotesAll_join['Ticker'].shift(-30)

In [8]:
quotesAll_join.head()

,100d,200d,21d,50d,9d,Adj Close,Close,Date,High,Low,...,EarningsDate,EarningsTime,5DayFuturePrice,5DayFutureTicker,10DayFuturePrice,10DayFutureTicker,20DayFuturePrice,20DayFutureTicker,30DayFuturePrice,30DayFutureTicker
0,NaN,NaN,NaN,NaN,NaN,28.210024,29.549999,NaN,29.559999,28.860001,...,NaN,NaN,25.422435,GGP,24.267304,GGP,24.926016,GGP,25.358830,GGP
1,NaN,NaN,NaN,NaN,NaN,27.398570,28.700001,NaN,29.080000,28.420000,...,NaN,NaN,25.622913,GGP,23.980909,GGP,24.219572,GGP,25.983905,GGP
2,NaN,NaN,NaN,NaN,NaN,27.379476,28.680000,NaN,28.830000,28.559999,...,NaN,NaN,25.527447,GGP,24.267304,GGP,24.181385,GGP,25.801190,GGP
3,NaN,NaN,NaN,NaN,NaN,27.532221,28.840000,NaN,29.049999,28.570000,...,NaN,NaN,25.174226,GGP,24.305489,GGP,25.174226,GGP,25.214580,GGP
4,NaN,NaN,NaN,NaN,NaN,26.252984,27.500000,NaN,28.420000,27.320000,...,NaN,NaN,24.038187,GGP,24.620527,GGP,24.945108,GGP,25.185731,GGP


### Subset DF

In [9]:
# Subset to include rows that have 30 day future prices
quotesAll_join2 = quotesAll_join[quotesAll_join['Ticker'] == quotesAll_join['30DayFutureTicker']]
# Subset to include rows (days) with actual trades
## This captures all our features on the day of the trade
## Additionall we have the future prices
tradesSub = quotesAll_join2[quotesAll_join2['Strike'].notnull()]

In [10]:
tradesSub.shape

(11178, 43)

In [11]:
tradesSub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11178 entries, 173 to 407495
Data columns (total 43 columns):
100d                 11146 non-null float64
200d                 8341 non-null float64
21d                  11178 non-null float64
50d                  11171 non-null float64
9d                   11178 non-null float64
Adj Close            11178 non-null float64
Close                11178 non-null float64
Date                 0 non-null float64
High                 11178 non-null float64
Low                  11178 non-null float64
MACD                 11178 non-null float64
Open                 11178 non-null float64
RSI                  11178 non-null float64
Volume               11178 non-null int64
lower_band           11178 non-null float64
max                  3088 non-null float64
min                  2888 non-null float64
upper_band           11178 non-null float64
TradeDate            11178 non-null object
Ticker               11178 non-null object
Strike             

### Drop Extra Columns
We Will drop some extra columns that will not impact the model building

In [12]:
subColumns = ['Ticker','Adj Close','100d','200d','21d','50d','9d','MACD','RSI',
                'Volume','OptionType','ActivityType','lower_band','upper_band',
                '5DayFuturePrice','10DayFuturePrice','20DayFuturePrice','30DayFuturePrice']

In [13]:
tradesSub = tradesSub[subColumns]

In [15]:
tradesSub.head()

,Ticker,Adj Close,100d,200d,21d,50d,9d,MACD,RSI,Volume,OptionType,ActivityType,lower_band,upper_band,5DayFuturePrice,10DayFuturePrice,20DayFuturePrice,30DayFuturePrice
173,GGP,30.065156,27.48,NaN,28.32,27.38,29.37,0.792183,77.733446,4168000,Calls,BUYING,26.369480,30.429078,30.250202,30.970909,30.172288,28.467914
174,GGP,30.065156,27.48,NaN,28.32,27.38,29.37,0.792183,77.733446,4168000,Calls,BUYING,26.369480,30.429078,30.289160,31.136477,30.133332,28.438698
178,GGP,30.250202,27.67,NaN,28.94,27.55,29.94,0.831434,76.208494,1999900,Calls,SELLING,27.005653,31.029444,30.970909,31.029346,29.217839,28.662701
180,GGP,30.483945,27.75,NaN,29.22,27.62,30.21,0.816642,78.158606,2838000,Puts,BUYING,27.344427,31.231191,30.561859,30.269681,29.013315,28.380261
182,GGP,30.844299,27.82,NaN,29.50,27.74,30.37,0.827390,80.884364,3108900,Puts,BUYING,27.671133,31.466760,31.117000,30.250202,28.399740,28.526351


### Create Categorical Price Outcome Columns
Now, we will create several different columns that correspond to the relationship between our response variable (future price).  This is for modeling purposes.

In [18]:
tradesSub['5DayFuturePriceCat'] = np.where(tradesSub['5DayFuturePrice']>tradesSub['Adj Close'], 1, 0)
tradesSub['10DayFuturePriceCat'] = np.where(tradesSub['10DayFuturePrice']>tradesSub['Adj Close'], 1, 0)
tradesSub['20DayFuturePriceCat'] = np.where(tradesSub['20DayFuturePrice']>tradesSub['Adj Close'], 1, 0)
tradesSub['30DayFuturePriceCat'] = np.where(tradesSub['30DayFuturePrice']>tradesSub['Adj Close'], 1, 0)


In [19]:
tradesSub.head()

,Ticker,Adj Close,100d,200d,21d,50d,9d,MACD,RSI,Volume,...,lower_band,upper_band,5DayFuturePrice,10DayFuturePrice,20DayFuturePrice,30DayFuturePrice,5DayFuturePriceCat,10DayFuturePriceCat,20DayFuturePriceCat,30DayFuturePriceCat
173,GGP,30.065156,27.48,NaN,28.32,27.38,29.37,0.792183,77.733446,4168000,...,26.369480,30.429078,30.250202,30.970909,30.172288,28.467914,1,1,1,0
174,GGP,30.065156,27.48,NaN,28.32,27.38,29.37,0.792183,77.733446,4168000,...,26.369480,30.429078,30.289160,31.136477,30.133332,28.438698,1,1,1,0
178,GGP,30.250202,27.67,NaN,28.94,27.55,29.94,0.831434,76.208494,1999900,...,27.005653,31.029444,30.970909,31.029346,29.217839,28.662701,1,1,0,0
180,GGP,30.483945,27.75,NaN,29.22,27.62,30.21,0.816642,78.158606,2838000,...,27.344427,31.231191,30.561859,30.269681,29.013315,28.380261,1,0,0,0
182,GGP,30.844299,27.82,NaN,29.50,27.74,30.37,0.827390,80.884364,3108900,...,27.671133,31.466760,31.117000,30.250202,28.399740,28.526351,1,0,0,0


### Create Percentage Difference Columns
Now, we will create columns reflecting the percentage difference between several of our technical indicator columns and our 'Adj Close' column

In [24]:
tradesSub['lower_band_pctdelt'] = (tradesSub['Adj Close'] - tradesSub['lower_band'])/tradesSub['lower_band']
tradesSub['upper_band_pctdelt'] = (tradesSub['Adj Close'] - tradesSub['upper_band'])/tradesSub['upper_band']
tradesSub['200d_pctdelt'] = (tradesSub['Adj Close'] - tradesSub['200d'])/tradesSub['200d']
tradesSub['100d_pctdelt'] = (tradesSub['Adj Close'] - tradesSub['100d'])/tradesSub['100d']
tradesSub['50d_pctdelt'] = (tradesSub['Adj Close'] - tradesSub['50d'])/tradesSub['50d']
tradesSub['21d_pctdelt'] = (tradesSub['Adj Close'] - tradesSub['21d'])/tradesSub['21d']
tradesSub['9d_pctdelt'] = (tradesSub['Adj Close'] - tradesSub['9d'])/tradesSub['9d']

In [27]:
tradesSub.head()

,Ticker,Adj Close,100d,200d,21d,50d,9d,MACD,RSI,Volume,...,10DayFuturePriceCat,20DayFuturePriceCat,30DayFuturePriceCat,lower_band_pctdelt,upper_band_pctdelt,100d_pctdelt,200d_pctdelt,50d_pctdelt,21d_pctdelt,9d_pctdelt
173,GGP,30.065156,27.48,NaN,28.32,27.38,29.37,0.792183,77.733446,4168000,...,1,1,0,0.140150,-0.011960,0.094074,NaN,0.098070,0.061623,0.023669
174,GGP,30.065156,27.48,NaN,28.32,27.38,29.37,0.792183,77.733446,4168000,...,1,1,0,0.140150,-0.011960,0.094074,NaN,0.098070,0.061623,0.023669
178,GGP,30.250202,27.67,NaN,28.94,27.55,29.94,0.831434,76.208494,1999900,...,1,0,0,0.120143,-0.025113,0.093249,NaN,0.098011,0.045273,0.010361
180,GGP,30.483945,27.75,NaN,29.22,27.62,30.21,0.816642,78.158606,2838000,...,0,0,0,0.114814,-0.023926,0.098521,NaN,0.103691,0.043256,0.009068
182,GGP,30.844299,27.82,NaN,29.50,27.74,30.37,0.827390,80.884364,3108900,...,0,0,0,0.114674,-0.019782,0.108710,NaN,0.111907,0.045569,0.015617


### Log-Scale Numeric Variables

In [61]:
# numColumns = ['Adj Close','100d','200d','21d','50d','9d','MACD','RSI',
#               'lower_band','upper_band','5DayFuturePrice','10DayFuturePrice',
#              '20DayFuturePrice','30DayFuturePrice']

In [28]:
# from sklearn import preprocessing
# import numpy as np

# for i in numColumns:
# #     tradesSub[i] = preprocessing.scale(tradesSub[i])
#     tradesSub[i] = (tradesSub[i] - tradesSub[i].mean())/tradesSub[i].std(ddof=0)
    
# tradesSub[numColumns] = preprocessing.scale(tradesSub[numColumns])

# dfTest[['A', 'B']] = scaler.fit_transform(dfTest[['A', 'B']])

In [31]:
import os
os.chdir("/Users/Collier/Dropbox/Skills/Python/Projects/Stocks/StockMarket-ML/")
tradesSub.to_csv("model_features.csv", sep='\t', encoding='utf-8')